---

_You are currently looking at **version 1.1** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 3

In this assignment you will explore text message data and create models to predict if a message is spam or not. 

In [78]:
import pandas as pd
import numpy as np

spam_data = pd.read_csv('spam.csv')

spam_data['target'] = np.where(spam_data['target']=='spam',1,0)
spam_data.head(10)

,text,target
0,"Go until jurong point, crazy.. Available only ...",0
1,Ok lar... Joking wif u oni...,0
2,Free entry in 2 a wkly comp to win FA Cup fina...,1
3,U dun say so early hor... U c already then say...,0
4,"Nah I don't think he goes to usf, he lives aro...",0
5,FreeMsg Hey there darling it's been 3 week's n...,1
6,Even my brother is not like to speak with me. ...,0
7,As per your request 'Melle Melle (Oru Minnamin...,0
8,WINNER!! As a valued network customer you have...,1
9,Had your mobile 11 months or more? U R entitle...,1


In [79]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(spam_data['text'], 
                                                    spam_data['target'], 
                                                    random_state=0)

### Question 1
What percentage of the documents in `spam_data` are spam?

*This function should return a float, the percent value (i.e. $ratio * 100$).*

In [80]:
def answer_one():
    
    
    return ((spam_data[spam_data['target']==1]).shape[0])/spam_data.shape[0]*100
#     return spam_data.shape[0]

In [81]:
answer_one()

13.406317300789663

### Question 2

Fit the training data `X_train` using a Count Vectorizer with default parameters.

What is the longest token in the vocabulary?

*This function should return a string.*

In [82]:
from sklearn.feature_extraction.text import CountVectorizer

def answer_two():
    cvect = CountVectorizer().fit(X_train)
    return max(cvect.get_feature_names(),key=lambda x:len(x))

In [83]:
answer_two()

'com1win150ppmx3age16subscription'

### Question 3

Fit and transform the training data `X_train` using a Count Vectorizer with default parameters.

Next, fit a fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1`. Find the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [84]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import roc_auc_score

def answer_three():
    cvect = CountVectorizer()
    x_train_cvect=cvect.fit_transform(X_train)
    x_test_cvect=cvect.transform(X_test)
    naive_bayes = MultinomialNB(alpha=0.1).fit(x_train_cvect,y_train)
    return roc_auc_score(y_test,naive_bayes.predict(x_test_cvect))

In [85]:
answer_three()

0.97208121827411165

### Question 4

Fit and transform the training data `X_train` using a Tfidf Vectorizer with default parameters.

What 20 features have the smallest tf-idf and what 20 have the largest tf-idf?

Put these features in a two series where each series is sorted by tf-idf value and then alphabetically by feature name. The index of the series should be the feature name, and the data should be the tf-idf.

The series of 20 features with smallest tf-idfs should be sorted smallest tfidf first, the list of 20 features with largest tf-idfs should be sorted largest first. 

*This function should return a tuple of two series
`(smallest tf-idfs series, largest tf-idfs series)`.*

In [86]:
from sklearn.feature_extraction.text import TfidfVectorizer
import operator
def answer_four():
    
    tfid=TfidfVectorizer()
    x_train_tfid = tfid.fit_transform(X_train)
    feat_names = np.array(tfid.get_feature_names())
    tfidf_ind=x_train_tfid.max(0).toarray()[0].argsort()
    tfidf_v=np.sort(x_train_tfid.max(0).toarray()[0])
    small_lst=np.array(sorted([[x,y] for x,y in zip(feat_names[tfidf_ind[:20]],tfidf_v[:20])], key=lambda x: (x[1], x[0])))
    large_lst=np.array(sorted([[x,y] for x,y in zip(feat_names[tfidf_ind[-21:-1]],tfidf_v[-21:-1])], key=lambda x: (-x[1], x[0])))
    small_sers=pd.Series(small_lst[:,1].astype(float),index=small_lst[:,0])
    large_sers=pd.Series(large_lst[:,1].astype(float),index=large_lst[:,0])
    return (small_sers,large_sers)


In [87]:
answer_four()

(aaniye          0.074475
 athletic        0.074475
 chef            0.074475
 companion       0.074475
 courageous      0.074475
 dependable      0.074475
 determined      0.074475
 exterminator    0.074475
 healer          0.074475
 listener        0.074475
 organizer       0.074475
 pest            0.074475
 psychiatrist    0.074475
 psychologist    0.074475
 pudunga         0.074475
 stylist         0.074475
 sympathetic     0.074475
 venaam          0.074475
 diwali          0.091250
 mornings        0.091250
 dtype: float64, 645         1.000000
 anything    1.000000
 anytime     1.000000
 beerage     1.000000
 done        1.000000
 er          1.000000
 havent      1.000000
 home        1.000000
 lei         1.000000
 nite        1.000000
 ok          1.000000
 okie        1.000000
 thank       1.000000
 thanx       1.000000
 too         1.000000
 where       1.000000
 yup         1.000000
 tick        0.980166
 blank       0.932702
 same        0.932467
 dtype: float64)

### Question 5

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **3**.

Then fit a multinomial Naive Bayes classifier model with smoothing `alpha=0.1` and compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [88]:
def answer_five():
    
    tfidf=TfidfVectorizer(min_df=3)
    X_train_vect = tfidf.fit_transform(X_train)
    naive_bayes = MultinomialNB(alpha=0.1).fit(X_train_vect,y_train)
    X_test_trans = tfidf.transform(X_test)
    return roc_auc_score(y_test,naive_bayes.predict(X_test_trans))

In [89]:
answer_five()

0.94162436548223349

### Question 6

What is the average length of documents (number of characters) for not spam and spam documents?

*This function should return a tuple (average length not spam, average length spam).*

In [90]:
def answer_six():

    all_y = np.concatenate((y_train,y_test),axis=0)
    all_x = np.concatenate((X_train,X_test),axis=0)
    avg_spam=[]
    avg_non_spam=[]
    for i in range(len(all_x)):
        if (all_y[i]):
            avg_spam.append(all_x[i])
        else:
            avg_non_spam.append(all_x[i])
    
    return (np.mean([len(x) for x in avg_non_spam]),np.mean([len(x) for x in avg_spam]))

In [91]:
answer_six()

(71.023626943005183, 138.8661311914324)

<br>
<br>
The following function has been provided to help you combine new features into the training data:

In [92]:
def add_feature(X, feature_to_add):
    """
    Returns sparse feature matrix with added feature.
    feature_to_add can also be a list of features.
    """
    from scipy.sparse import csr_matrix, hstack
    return hstack([X, csr_matrix(feature_to_add).T], 'csr')

### Question 7

Fit and transform the training data X_train using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5**.

Using this document-term matrix and an additional feature, **the length of document (number of characters)**, fit a Support Vector Classification model with regularization `C=10000`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [93]:
from sklearn.svm import SVC

def answer_seven():
    tfidf = TfidfVectorizer(min_df=5)
    X_train_tfidf = tfidf.fit_transform(X_train)
    X_train_tfidf=add_feature(X_train_tfidf,X_train.str.len())
    X_test_trans = tfidf.transform(X_test)
    X_test_trans = add_feature(X_test_trans,X_test.str.len())
    svc = SVC(C=10000)
    svc.fit(X_train_tfidf,y_train)
    pred=svc.predict(X_test_trans)
    
    
    return roc_auc_score(y_test,pred)

In [94]:
answer_seven()

0.95813668234215565

### Question 8

What is the average number of digits per document for not spam and spam documents?

*This function should return a tuple (average # digits not spam, average # digits spam).*

In [95]:
import re
def answer_eight():
    digits=re.compile(r'[0-9]')
    spam=[len(re.findall(digits,x)) for x in spam_data.loc[spam_data['target']==1,'text']]
    not_spam=[len(re.findall(digits,x)) for x in spam_data.loc[spam_data['target']==0,'text']]
#     spam=spam_data.loc[spam_data['target']==1,'text'].applyre.findall(digits,).apply(len)
#     not_spam=spam_data.loc[spam_data['target']==0,'text']
    
    return (np.mean(not_spam),np.mean(spam))

In [96]:
answer_eight()

(0.29927461139896372, 15.759036144578314)

### Question 9

Fit and transform the training data `X_train` using a Tfidf Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **word n-grams from n=1 to n=3** (unigrams, bigrams, and trigrams).

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* **number of digits per document**

fit a Logistic Regression model with regularization `C=100`. Then compute the area under the curve (AUC) score using the transformed test data.

*This function should return the AUC score as a float.*

In [111]:
from sklearn.linear_model import LogisticRegression

def answer_nine():
    digits=re.compile(r'[0-9]')
    tfidf=TfidfVectorizer(min_df=5,ngram_range=(1,3))
    X_train_trans = tfidf.fit_transform(X_train)
    X_train_trans= add_feature(X_train_trans,X_train.str.len())
    X_train_trans=add_feature(X_train_trans,[len(re.findall(digits,x)) for x in X_train])
    X_test_trans = tfidf.transform(X_test)
    X_test_trans=add_feature(X_test_trans,X_test.str.len())
    X_test_trans=add_feature(X_test_trans,[len(re.findall(digits,x)) for x in X_test])
    
    logreg = LogisticRegression(C=100)
    logreg.fit(X_train_trans,y_train)
    return roc_auc_score(y_test,logreg.predict(X_test_trans))

In [112]:
answer_nine()

0.96787090640544626

### Question 10

What is the average number of non-word characters (anything other than a letter, digit or underscore) per document for not spam and spam documents?

*Hint: Use `\w` and `\W` character classes*

*This function should return a tuple (average # non-word characters not spam, average # non-word characters spam).*

In [120]:
def answer_ten():
    regex = re.compile('\W')
    spam=[len(re.findall(regex,x)) for x in spam_data.loc[spam_data['target']==1,'text']]
    not_spam=[len(re.findall(regex,x)) for x in spam_data.loc[spam_data['target']==0,'text']]
    return (np.mean(not_spam),np.mean(spam))

In [121]:
answer_ten()

(17.291813471502589, 29.041499330655956)

### Question 11

Fit and transform the training data X_train using a Count Vectorizer ignoring terms that have a document frequency strictly lower than **5** and using **character n-grams from n=2 to n=5.**

To tell Count Vectorizer to use character n-grams pass in `analyzer='char_wb'` which creates character n-grams only from text inside word boundaries. This should make the model more robust to spelling mistakes.

Using this document-term matrix and the following additional features:
* the length of document (number of characters)
* number of digits per document
* **number of non-word characters (anything other than a letter, digit or underscore.)**

fit a Logistic Regression model with regularization C=100. Then compute the area under the curve (AUC) score using the transformed test data.

Also **find the 10 smallest and 10 largest coefficients from the model** and return them along with the AUC score in a tuple.

The list of 10 smallest coefficients should be sorted smallest first, the list of 10 largest coefficients should be sorted largest first.

The three features that were added to the document term matrix should have the following names should they appear in the list of coefficients:
['length_of_doc', 'digit_count', 'non_word_char_count']

*This function should return a tuple `(AUC score as a float, smallest coefs list, largest coefs list)`.*

In [141]:
def answer_eleven():
    digits=re.compile(r'[0-9]')
    regex=re.compile(r'\W')
    cvect=CountVectorizer(min_df=5,ngram_range=(2,5),analyzer='char_wb')
    X_train_trans=cvect.fit_transform(X_train)
    X_train_trans= add_feature(X_train_trans,X_train.str.len())
    X_train_trans=add_feature(X_train_trans,[len(re.findall(digits,x)) for x in X_train])
    X_train_trans=add_feature(X_train_trans,[len(re.findall(regex,x)) for x in X_train])
    X_test_trans=cvect.transform(X_test)
    X_test_trans= add_feature(X_test_trans,X_test.str.len())
    X_test_trans=add_feature(X_test_trans,[len(re.findall(digits,x)) for x in X_test])
    X_test_trans=add_feature(X_test_trans,[len(re.findall(regex,x)) for x in X_test])
    logreg = LogisticRegression(C=100)
    logreg.fit(X_train_trans,y_train)
    pred=logreg.predict(X_test_trans)
    feature_names = np.array(cvect.get_feature_names())
    sorted_coef_index = logreg.coef_[0].argsort()
    return (roc_auc_score(y_test,pred).astype(float),feature_names[sorted_coef_index[:10]],feature_names[sorted_coef_index[-11:-1]][::-1])

In [142]:
answer_eleven()

(0.97885931107074342,
 array(['. ', '..', '? ', ' i', ' y', ' go', ':)', ' h', 'go', ' m'], 
       dtype='<U5'),
 array(['ne', 'ia', 'co', 'xt', ' ch', 'mob', ' x', 'ww', 'ar', 'eply '], 
       dtype='<U5'))